In [9]:
cd manifests

/home/ubuntu/mlops_workshops/seldon/manifests


### Let's deploy a Housing Preprocessor that gets the data from the feature store and standardize it

In [ ]:
!kubectl apply -f seldondeployment1.yaml

In [ ]:
# in terminal: kubectl port-forward svc/seldon-preprocessor-model-model 9100:9000

In [31]:
import requests
response = requests.post('http://localhost:9100/api/v1.0/predictions', json={"data":{"ndarray":[[1], [5]]}}).json()
response

{'data': {'names': ['house_main_view:HouseId',
   'house_main_view:MedInc',
   'house_main_view:HouseAge',
   'house_main_view:AveRooms',
   'house_main_view:AveBedrms',
   'house_main_view:Population',
   'house_main_view:AveOccup',
   'house_lat_lon_view:Latitude',
   'house_lat_lon_view:Longitude'],
  'ndarray': [[-1.7319668924113674,
    -0.7324396877792114,
    -0.3686446776412593,
    -0.7979812453599388,
    0.07906991444149086,
    0.7289808981819601,
    0.08193376858427932,
    -0.6797418814172994,
    0.5938194097272528],
   [-1.7312955548882658,
    -1.3334053246300217,
    -0.8453931491070594,
    -0.34815492500568834,
    -0.019726266386578314,
    1.1316531272558914,
    -0.018082498363888314,
    1.6377819950507821,
    -0.9834354838678211]]},
 'meta': {'requestPath': {'model': 'dvoitekh/seldon_feast:workshop8'}}}

In [32]:
response['data']['ndarray']

[[-1.7319668924113674,
  -0.7324396877792114,
  -0.3686446776412593,
  -0.7979812453599388,
  0.07906991444149086,
  0.7289808981819601,
  0.08193376858427932,
  -0.6797418814172994,
  0.5938194097272528],
 [-1.7312955548882658,
  -1.3334053246300217,
  -0.8453931491070594,
  -0.34815492500568834,
  -0.019726266386578314,
  1.1316531272558914,
  -0.018082498363888314,
  1.6377819950507821,
  -0.9834354838678211]]

In [33]:
!kubectl delete -f seldondeployment1.yaml

seldondeployment.machinelearning.seldon.io "seldon-preprocessor" deleted


### Let's deploy a model that consumes this data

In [10]:
!kubectl apply -f seldondeployment2.yaml

seldondeployment.machinelearning.seldon.io/seldon-regressor created


In [ ]:
# in terminal: kubectl port-forward svc/seldon-regressor-model-model 9200:9000

In [36]:
response = requests.post('http://localhost:9200/api/v1.0/predictions', json={"data":{"ndarray":response['data']['ndarray']}}).json()
response

{'data': {'names': [], 'ndarray': [2.0, 0.596]},
 'meta': {'requestPath': {'model': 'dvoitekh/seldon_feast:workshop9'}}}

In [ ]:
!kubectl delete -f seldondeployment2.yaml

### Let's deploy them together!

In [ ]:
!kubectl apply -f seldondeployment3.yaml

In [ ]:
!kubectl get po

In [ ]:
!kubectl get svc

In [ ]:
# in terminal: kubectl port-forward svc/seldon-pipeline-model 8100:8000

In [38]:
response = requests.post('http://localhost:8100/api/v1.0/predictions', json={"data":{"ndarray":[[2], [5]]}}).json()
response

{'data': {'names': [], 'ndarray': [2.017, 0.596]},
 'meta': {'requestPath': {'preprocessor': 'dvoitekh/seldon_feast:workshop9',
   'regressor': 'dvoitekh/seldon_feast:workshop9'}}}